<a href="https://colab.research.google.com/github/RPGraciotti/BootCampAlura/blob/main/Modulo_5/Aula_3_aleatoriedade_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preparando conteúdo anterior

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

dados = pd.read_excel("https://github.com/alura-cursos/covid-19-clinical/blob/main/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")
### também posso usar o link no meu repositório

def preenche_tabela(dados):
    features_continuas_colunas = dados.iloc[:, 13:-2].columns
    features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:, -2:]
    dados_finais = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
    dados_finais.columns = dados.columns
    return dados_finais

dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()

def prepare_window(rows):
    if(np.any(rows["ICU"])):
        rows.loc[rows["WINDOW"]=="0-2", "ICU"] = 1
    return rows.loc[rows["WINDOW"] == "0-2"]

dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
dados_limpos.AGE_PERCENTIL = dados_limpos.AGE_PERCENTIL.astype("category").cat.codes # transformando em variaveis categoricas


np.random.seed(73246)

x_columns = dados.columns
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU","WINDOW"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
modelo = DummyClassifier()
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.48863636363636365

In [2]:
modelo = LogisticRegression(max_iter=10000)
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

0.6818181818181818

In [3]:
modelo_arvore = DecisionTreeClassifier()

modelo_arvore.fit(x_train, y_train)
pred_arvore = modelo_arvore.predict(x_test)
accuracy_score(y_test, pred_arvore)


0.6590909090909091

# Aula 3

In [4]:

def roda_n_modelos(modelo, dados, n):

  x_columns = dados.columns
  y = dados["ICU"]
  x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)
  auc_lista = []

  for _ in range(n):

    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
    modelo.fit(x_train, y_train)
    prob_predic = modelo.predict_proba(x_test)
    auc = roc_auc_score(y_test, prob_predic[:,1])
    auc_lista.append(auc)

  auc_medio = np.mean(auc_lista)
  auc_std = np.std(auc_lista)
  print(f"AUC {auc_medio}")
  print(f"Intervalo {auc_medio - 2 * auc_std} - {auc_medio + 2 * auc_std}") # confidence interval
  #plt.hist(auc_lista)

In [5]:
roda_n_modelos(modelo_arvore, dados = dados_limpos, n = 100)

AUC 0.6290710949662688
Intervalo 0.5455964028876362 - 0.7125457870449013


In [6]:
roda_n_modelos(modelo, dados = dados_limpos, n = 100)

AUC 0.7581733264141151
Intervalo 0.6716374311886243 - 0.8447092216396058


# Desafio 01 - Estudar a documentação do sklearn e testar outros modelos

Um tipo de algoritmo que me interessou enquanto estudava modelos básicos de machine learning é o modelo de Naive Bayes, que utiliza preceitos de estatística bayesiana para gerar classificações. Sem entrar muito a fundo nos ajustes do modelo, vamos realizar um teste simples de Naive Bayes usando os nossos métodos aplicados anteriormente. Utilizarei o algoritmos de Naive Bayes Gaussiano e multinomial, a forma mais básica desse tipo de algoritmo.

In [7]:
from sklearn.naive_bayes import GaussianNB

In [8]:
gnb = GaussianNB()

roda_n_modelos(modelo = gnb, dados = dados_limpos, n = 1000)

AUC 0.7318051375194602
Intervalo 0.6411485907821692 - 0.8224616842567511


Esse tipo de modelo é muito utilizado em classificações binárias, e parece dar um resultado interessante de alta acurácia. Reforço aqui que é apenas uma exploração da facilidade de aplicação de modelos, sem entrar em muitos detalhes sobre a real melhora ou adequabiliade de um modelo em relação ao outro.